# Xarray - Pandas converter
---------------------------

This example is the example used in Xarray user-guide (section ["working with pandas"](https://docs.xarray.dev/en/stable/user-guide/pandas.html)).

A more complete example is in the [other Notebook](https://nbviewer.org/github/loco-philippe/ntv-pandas/blob/main/example/example_multidimensional.ipynb).

## Xarray interface

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import ntv_pandas # activate pandas npd accessor
import ntv_numpy  # activate xarray nxr accessor

ds = xr.Dataset(
    {"foo": (("x", "y"), np.random.randn(2, 3))},
    coords={
        "x": [10, 20],
        "y": ["a", "b", "c"],
        "along_x": ("x", np.random.randn(2)),
        "scalar": 123,
    },
    attrs={"example": "Xarray user-guide"}
)
ds

<xarray.Dataset> Size: 88B
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 8B 10 20
  * y        (y) <U1 12B 'a' 'b' 'c'
    along_x  (x) float64 16B 0.3947 -0.05511
    scalar   int32 4B 123
Data variables:
    foo      (x, y) float64 48B -0.1768 0.3231 -0.4494 1.791 1.605 -1.376
Attributes:
    example:  Xarray user-guide

*Note:*

- the `attrs` metadata is an addition to the example in the Xarray user-guide.

In [2]:
df = ds.to_dataframe()
df

foo   along_x  scalar
x  y                            
10 a -0.176827  0.394717     123
   b  0.323146  0.394717     123
   c -0.449401  0.394717     123
20 a  1.791361 -0.055112     123
   b  1.604509 -0.055112     123
   c -1.376154 -0.055112     123

In [3]:
xr.Dataset.from_dataframe(df)

<xarray.Dataset> Size: 152B
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 8B 10 20
  * y        (y) object 24B 'a' 'b' 'c'
Data variables:
    foo      (x, y) float64 48B -0.1768 0.3231 -0.4494 1.791 1.605 -1.376
    along_x  (x, y) float64 48B 0.3947 0.3947 0.3947 -0.05511 -0.05511 -0.05511
    scalar   (x, y) int32 24B 123 123 123 123 123 123

This example shows that the conversion is not reversible (lossy roundtrip) and that the size of the ``dataset`` increases.

Particularly after a roundtrip, the following deviations are noted:

- a non-dimension Dataset ``coordinate`` is converted into ``variable`` 
- a non-dimension DataArray ``coordinate`` is not converted 
- ``dtype`` is not allways the same (e.g. "str" is converted to "object")
- ``attrs`` metadata is not converted

The `ntv_pandas` converter avoids these data loss as explained below.

## ntv_pandas converter : Dataset -> DataFrame

Three options are available :

- **ntv_type**: Boolean (default True) - if False the `ntv_type` is not included in the columns name
- **info**: Boolean (default True) - if True, the `DataFrame.attrs` contains the multidimensional structure
- **index**: Boolean (default True) - if True, dimensions are translated into `indexes`

In [4]:
df_min = ds.nxr.to_dataframe(ntv_type=False, info=False, index=False)  # without additional data
df_min

,x,y,along_x,foo,scalar
0,10,a,0.394717,-0.176827,123
1,10,b,0.394717,0.323146,123
2,10,c,0.394717,-0.449401,123
3,20,a,-0.055112,1.791361,123
4,20,b,-0.055112,1.604509,123
5,20,c,-0.055112,-1.376154,123


In [5]:
df_min.attrs

{}

In [6]:
df_full = ds.nxr.to_dataframe()
df_full

along_x:float64  foo:float64  scalar:int32
x:int32 y:string                                            
10      a                0.394717    -0.176827           123
        b                0.394717     0.323146           123
        c                0.394717    -0.449401           123
20      a               -0.055112     1.791361           123
        b               -0.055112     1.604509           123
        c               -0.055112    -1.376154           123

In [7]:
df_full.attrs

{'info': {'dimensions': ['x', 'y'],
  'data': {'example': {'meta': 'Xarray user-guide', 'xtype': 'meta'},
   'x': {'shape': [2], 'xtype': 'namedarray'},
   'y': {'shape': [3], 'xtype': 'namedarray'},
   'along_x': {'shape': [2], 'xtype': 'variable', 'links': ['x']},
   'scalar': {'shape': [1], 'xtype': 'namedarray'},
   'foo': {'shape': [2, 3], 'xtype': 'variable', 'links': ['x', 'y']}}},
 'metadata': {'example': 'Xarray user-guide'}}

*note*:

- The `DataFrame.attrs` attribute is still experimental (some operations remove it). The associated information must therefore be processed as a priority.

## ntv_pandas converter : DataFrame -> Dataset

The conversion is done without loss, by reading the `DataFrame.attrs` or by finding the multidimensional structure hidden by the tabular structure.

Three options are available:

- **dims**: list of string (default None) - order of dimensions to apply
- **dataset** : Boolean (default True) - if False and a single data_var,
return a xr.DataArray
- **info** : Boolean (default True) - if True, use `DataFrame.attrs`

In [8]:
ds_min = df_min.npd.to_xarray()
ds_min

<xarray.Dataset> Size: 88B
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 8B 10 20
  * y        (y) <U1 12B 'a' 'b' 'c'
    along_x  (x) float64 16B 0.3947 -0.05511
    scalar   int32 4B 123
Data variables:
    foo      (x, y) float64 48B -0.1768 0.3231 -0.4494 1.791 1.605 -1.376

*Note:*

- The multidimensional structure is found by the `tab_analysis` module

In [9]:
df_min.reset_index(drop=False).npd.analysis(distr=True).field_partition()

{'primary': ['x', 'y'],
 'secondary': ['along_x'],
 'mixte': [],
 'unique': ['scalar'],
 'variable': ['index', 'foo']}

In [10]:
ds_full = df_full.npd.to_xarray()
ds_full

<xarray.Dataset> Size: 88B
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 8B 10 20
  * y        (y) <U1 12B 'a' 'b' 'c'
    along_x  (x) float64 16B 0.3947 -0.05511
    scalar   int32 4B 123
Data variables:
    foo      (x, y) float64 48B -0.1768 0.3231 -0.4494 1.791 1.605 -1.376
Attributes:
    example:  Xarray user-guide

*Note:*

- The multidimensional structure is preserved with both options
- The `dtype` is preserved with both options 